# Visualizing Group Structure

**Module 01f** | Modular Arithmetic and Groups

*A single picture can reveal structure that tables of numbers hide.*

> **Question:** You've computed orders, generators, subgroups, and cosets with numbers and equations. But can you *see* them? What does a cyclic group actually *look like*?

## Objectives

By the end of this notebook you will be able to:

1. Draw **Cayley graphs** for small groups and interpret their structure
2. Construct and read a **subgroup lattice** diagram
3. Use **multiplication table heatmaps** to spot symmetry and subgroup boundaries

## From Symbols to Pictures

In [01a](01a-integers-and-division.ipynb)–[01e](01e-subgroups-lagrange.ipynb) everything was symbolic: equations, tables, lists. Now we make it **visual**. You'll literally *see* why generators reach everywhere, how subgroups nest, and where symmetry hides.

## Cayley Graphs

A **Cayley graph** represents a group as a picture:
- **Nodes** = group elements (one node per element)
- **Edges** = the action of a chosen element $g$: draw an arrow from $a$ to $a + g$ (additive) or $a \cdot g$ (multiplicative) for every element $a$

If $g$ is a generator, following the arrows visits every node before returning to the start. If $g$ is not a generator, the arrows break into disconnected loops. Let's see both cases in $(\mathbb{Z}/6\mathbb{Z}, +)$.

In [ ]:
# Cayley graph of (Z/6Z, +) with generator 1
R = Zmod(6)
g = R(1)

print(f'Cayley graph of (Z/6Z, +) with generator g = {int(g)}.')
print(f'Rule: draw an arrow from a to a + {int(g)} (mod 6) for every a.\n')

G_graph = DiGraph()
for a in R:
    target = a + g
    print(f'  {int(a)} -> {int(a)} + {int(g)} = {int(target)}')
    G_graph.add_edge(int(a), int(target))

G_graph.plot(layout='circular', vertex_size=800, vertex_color='lightblue',
       edge_color='steelblue', figsize=5).show()
print(f'One connected cycle visiting all 6 elements.')
print(f'Generator 1 reaches everything, just like we proved in 01d.')

One connected cycle. Following the arrows from any node, you visit all 6 elements and return to the start. That is what "generator" looks like as a picture.

Now let's try element 2. Since $\gcd(2, 6) = 2 \neq 1$, element 2 is NOT a generator ([01d](01d-cyclic-groups-generators.ipynb)). What does the Cayley graph look like?

In [ ]:
# Cayley graph of (Z/6Z, +) with non-generator 2
R = Zmod(6)
g = R(2)

print(f'Cayley graph of (Z/6Z, +) with g = {int(g)} (not a generator).')
print(f'Rule: draw an arrow from a to a + {int(g)} (mod 6) for every a.\n')

G_graph = DiGraph()
for a in R:
    target = a + g
    print(f'  {int(a)} -> {int(a)} + {int(g)} = {int(target)}')
    G_graph.add_edge(int(a), int(target))

print(f'\nLook: 0 -> 2 -> 4 -> 0 forms one loop. 1 -> 3 -> 5 -> 1 forms another.')
print(f'Element 2 never crosses from evens to odds.\n')

G_graph.plot(layout='circular', vertex_size=800, vertex_color='lightblue',
       edge_color='red', figsize=5).show()
d = gcd(int(g), 6)
print(f'gcd({int(g)}, 6) = {d}.')
print(f'The graph splits into {d} disconnected cycles, each of length 6/{d} = {6//d}.')
print(f'This is the visual meaning of "not a generator": the arrows cannot reach everyone.')

> **Common mistake:** "A Cayley graph shows the group." More precisely, a Cayley graph depends on the *choice of generator*, not just the group. The same group $\mathbb{Z}/6\mathbb{Z}$ looks like a hexagon with generator 1, but decomposes into two triangles with generator 2.

## Subgroup Lattice Diagrams

In [01e](01e-subgroups-lagrange.ipynb) we found every subgroup of a group and verified that their sizes divide the group size. A **subgroup lattice** arranges all those subgroups in a diagram:

- Each **node** is a subgroup
- A **line going up** from $H_1$ to $H_2$ means $H_1 \subseteq H_2$ (every element of $H_1$ is also in $H_2$)
- The **bottom** is always the trivial subgroup $\{0\}$, the **top** is the whole group

For cyclic groups, the subgroup structure mirrors the **divisor lattice** of $n$. Let's build this step by step for $\mathbb{Z}/12\mathbb{Z}$.

In [ ]:
# Subgroup lattice of Z/12Z
n = 12
R = Zmod(n)
divs = divisors(n)

# Step 1: list every subgroup with its elements
print(f'Subgroups of (Z/{n}Z, +):')
print(f'One subgroup per divisor of {n}: {divs}\n')

for d in divs:
    gen = n // d
    elems = sorted(set(int(R(gen) * k) for k in range(d)))
    print(f'  <{gen}> = {elems}   (size {d})')

# Step 2: show which subgroups sit inside which (immediate containments)
print(f'\nContainment (H1 inside H2 when |H1| divides |H2|):')
for d1 in divs:
    containers = [d2 for d2 in divs if d2 != d1 and d2 % d1 == 0
                  and not any(d1 < d3 < d2 and d2 % d3 == 0 and d3 % d1 == 0 for d3 in divs)]
    if containers:
        print(f'  <{n//d1}> (size {d1})  inside  ' +
              ', '.join(f'<{n//d2}> (size {d2})' for d2 in containers))

# Step 3: draw the lattice
relations = [(d1, d2) for d1 in divs for d2 in divs if d1 != d2 and d2 % d1 == 0]
P = Poset((divs, relations))
P.plot(figsize=6, vertex_size=800, vertex_color='lightyellow',
       element_labels={d: f'<{n//d}>\n|{d}|' for d in divs}).show()
print(f'Each node: <generator> on top, |size| on bottom.')
print(f'Bottom = {{0}} (trivial). Top = all of Z/{n}Z.')
print(f'A line going up means "is contained in."')

The lattice mirrors the divisor lattice of 12. This is no coincidence: for cyclic groups, subgroups of $\mathbb{Z}/n\mathbb{Z}$ correspond exactly to divisors of $n$. The subgroup of size $d$ is $\langle n/d \rangle$.

Notice how the lattice captures containment at a glance. For example, $\langle 6 \rangle = \{0, 6\}$ (size 2) sits inside $\langle 3 \rangle = \{0, 3, 6, 9\}$ (size 4), which sits inside $\langle 1 \rangle = \{0, 1, \ldots, 11\}$ (size 12). Each step up means "the smaller subgroup is entirely contained in the larger one."

> **Checkpoint.** $\mathbb{Z}/8\mathbb{Z}$ has divisors $\{1, 2, 4, 8\}$. Predict what its subgroup lattice looks like before trying Exercise 2.

## Multiplication Table Heatmaps

A color-coded multiplication table can reveal structure that raw numbers hide. Let's start by looking at the actual table, then color it.

In [ ]:
# Multiplication table of (Z/7Z*, ×)
R = Zmod(7)
units = sorted(int(u) for u in R.list_of_elements_of_multiplicative_group())

print(f'Multiplication table of (Z/7Z*, x):')
print(f'Elements: {units}\n')

# Print the numeric table
header = '   x |' + ''.join(f'{u:>4}' for u in units)
print(header)
print('   --+' + '-' * (4 * len(units)))
for a in units:
    row = [int(R(a) * R(b)) for b in units]
    print(f'  {a:>2} |' + ''.join(f'{r:>4}' for r in row))

print(f'\nEvery row is a permutation of {units} (each value appears exactly once).')
print(f'The table is symmetric about the diagonal: a*b = b*a (the group is abelian).')
print(f'\nNow the same table as a heatmap (color = product value):')

M = matrix(ZZ, len(units), len(units),
           lambda i, j: int(R(units[i]) * R(units[j])))
matrix_plot(M, cmap='viridis', figsize=5, origin='upper').show()
print(f'Symmetric pattern = commutativity visible at a glance.')
print(f'Each color appears once per row and once per column (Latin square property).')

Does this pattern hold for other primes? Let's compare with a smaller group to see what changes and what stays the same.

In [ ]:
# Multiplication table of (Z/5Z*, ×) for comparison
R = Zmod(5)
units = sorted(int(u) for u in R.list_of_elements_of_multiplicative_group())

print(f'Multiplication table of (Z/5Z*, x):')
print(f'Elements: {units}\n')

header = '   x |' + ''.join(f'{u:>4}' for u in units)
print(header)
print('   --+' + '-' * (4 * len(units)))
for a in units:
    row = [int(R(a) * R(b)) for b in units]
    print(f'  {a:>2} |' + ''.join(f'{r:>4}' for r in row))

print(f'\nSame properties: Latin square (each value once per row/column), symmetric.')

M = matrix(ZZ, len(units), len(units),
           lambda i, j: int(R(units[i]) * R(units[j])))
matrix_plot(M, cmap='viridis', figsize=4, origin='upper').show()
print(f'Smaller group, different colors, but the same structural pattern.')
print(f'Both are cyclic abelian groups, so both have symmetric Latin square tables.')

## Exercises

### Exercise 1 (Worked)
Draw the Cayley graph of $(\mathbb{Z}/8\mathbb{Z}, +)$ with generator 3. Since $\gcd(3, 8) = 1$, it should visit all 8 elements. Then change to generator 2 and observe the difference.

In [ ]:
# Exercise 1 — Cayley graphs of Z/8Z
R = Zmod(8)
n = 8

for g_val, color, label in [(3, 'steelblue', 'generator 3'), (2, 'red', 'non-generator 2')]:
    g = R(g_val)
    print(f'=== Z/{n}Z with {label} ===\n')

    G_graph = DiGraph()
    for a in R:
        target = a + g
        print(f'  {int(a)} -> {int(a)} + {int(g)} = {int(target)}')
        G_graph.add_edge(int(a), int(target))

    d = gcd(g_val, n)
    print(f'\n  gcd({g_val}, {n}) = {d} -> {d} cycle(s) of length {n // d}')

    G_graph.plot(layout='circular', vertex_size=600, vertex_color='lightblue',
           edge_color=color, figsize=4, title=f'Z/{n}Z with {label}').show()
    print()

### Exercise 2 (Guided)
Compare the subgroup lattice of $\mathbb{Z}/12\mathbb{Z}$ and $\mathbb{Z}/8\mathbb{Z}$. Which has more subgroups? Why?

In [ ]:
# Exercise 2 — Compare subgroup lattices
# Z/12Z lattice (provided above)
# TODO: adapt the lattice code for Z/8Z
n = 8
divs = divisors(n)
print(f'Divisors of {n}: {divs}')
print(f'Number of subgroups: {len(divs)}')
# TODO: build and plot the Poset for Z/8Z
# TODO: compare with Z/12Z (which has divisors {1,2,3,4,6,12} = 6 subgroups)

### Exercise 3 (Independent)
Create a multiplication table heatmap for $(\mathbb{Z}/15\mathbb{Z}^*, \times)$. The group has $\varphi(15) = 8$ elements. Can you identify visual "blocks" in the heatmap? What subgroup do they correspond to?

In [ ]:
# Exercise 3 — Your code here


## Summary

| Concept | Key idea |
|---------|----------|
| **Cayley graphs** | Nodes are elements, edges show the action of a generator. Full connection means generator, disconnected components mean non-generator |
| **Subgroup lattices** | For cyclic groups, the subgroup structure mirrors the divisor lattice of $n$ |
| **Heatmaps** | Color-coded multiplication tables make commutativity and subgroup boundaries visible as symmetry patterns |

Visualization turns abstract algebraic properties into spatial intuition. Generators that reach everything look connected, subgroups that nest look hierarchical, and commutativity shows up as symmetry about the diagonal.

> **Crypto teaser:** In elliptic curve cryptography, the "group" is a set of points on a curve, not numbers mod $p$. Visualizing EC group structure, and how it differs from $\mathbb{Z}/p\mathbb{Z}^*$, will be the subject of Module 06.

---

*This concludes the Explore phase of Module 01. Next steps:*
- **Implement**: Open `rust/src/lib.rs` and build `mod_exp`, `gcd`, `is_generator`, `element_order`, `find_all_generators` from scratch
- **Break**: Attack [smooth-order groups](../break/smooth-order-attack.ipynb) and [weak generators](../break/weak-generator-attack.ipynb)
- **Connect**: See these concepts in [RSA](../connect/rsa-key-generation.ipynb) and [Diffie-Hellman](../connect/dh-parameter-selection.ipynb)